In [6]:
import numpy as np
from utils.data_loading import read_csv
from modules.coincidence_counts import count_coincidences
from modules.coincidence_counts import find_closest_event_index

In [7]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams.update({'figure.max_open_warning': 0})
%matplotlib widget

# Measurement of the second order correlation function

In [8]:
sync, time = read_csv('measurements/g2_correlation_function/full_SinglePhoton_g2_workspace.csv')

In [9]:
from numba import jit

@jit(nopython=True)
def g2_heralded(time_arr, sync_arr, coinc_window, tau_max, tau_bin_size, gate_id=1, lead_id=2, lag_id=3):

    tau_arr = np.arange(-tau_max, tau_max+tau_bin_size, tau_bin_size)

    N_A = np.where(sync_arr == gate_id, 1, 0).sum()
    N_AB = count_coincidences(
        time_arr, sync_arr, coinc_window, channel_1_id=gate_id, channel_2_id=lead_id)
    N_AC = count_coincidences(
        time_arr, sync_arr, coinc_window, channel_1_id=gate_id, channel_2_id=lag_id)

    total_time = time_arr[-1]

    g2 = []

    for tau in tau_arr:

        coinc_counter = 0
        
        for index, event in enumerate(sync_arr):
            if event != gate_id:
                continue
                
            gate_event_time = time_arr[index]
            
            closest_lead_index, dt_lead = find_closest_event_index(time_arr, sync_arr, index, lead_id)

            if dt_lead > coinc_window/2:
                continue
                
            coinc_found = False
            
            # Search backwards (at times before Gate event).
            search_index = index - 1
            while search_index > 0 and gate_event_time - time_arr[search_index] <= coinc_window/2 + tau_max + tau_bin_size:
                if sync_arr[search_index] == lag_id:                    
                    if gate_event_time + tau - tau_bin_size/2 <= time_arr[search_index] <= gate_event_time + tau + tau_bin_size/2:
                        coinc_found = True
                        break
                        
                search_index -= 1
    
            
            # Search forward
            search_index = index + 1
            while search_index <= len(sync_arr) and time_arr[search_index] - gate_event_time <= coinc_window/2 + tau_max + tau_bin_size:
                if sync_arr[search_index] == lag_id:
                    if gate_event_time + tau - tau_bin_size/2 <= time_arr[search_index] <= gate_event_time + tau + tau_bin_size/2:
                        coinc_found = True
                        break
                    
                search_index += 1
            
            if coinc_found:
                coinc_counter += 1
                                
        g2.append(coinc_counter*N_A/(N_AB*N_AC)*total_time/coinc_window**2/tau_bin_size)

    return g2, tau_arr

In [10]:
coinc_window = 5000
g2, tau = g2_heralded(time, sync, coinc_window, 2*coinc_window, coinc_window, gate_id=1, lead_id=2, lag_id=3)

In [11]:
fig, ax = plt.subplots()
ax.plot(tau, g2)

ax.set_xlabel('tau [ps]')
ax.set_ylabel('g2(tau)') 

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Coherent State